In [1]:
import sys
sys.path.append('/Users/bhargavdesai/Desktop/IU/RA/CNS/hra-amap/code')

import trimesh

from organ import Organ
from tissue import TissueBlock
from pipeline import Pipeline

In [4]:
# load source and target organs
source = Organ(path='../data/Pancreas/Source/generic_pancreas_organ.glb')
target = Organ(path='../data/Pancreas/Reference/VH_F_Pancreas.glb')

In [12]:
# load source and reference organs
source_atlas = Organ(path='/Users/bhargavdesai/Desktop/IU/RA/CNS/VH_F_Spleen_v1.0.glb')
hra_atlas = Organ(path='/Users/bhargavdesai/Desktop/IU/RA/CNS/VH_F_Spleen_v1.2.glb')

concatenating texture: may result in visual artifacts


concatenating texture: may result in visual artifacts


In [6]:
source.show()

In [7]:
target.show()

In [9]:
# instantiate the registration pipeline
pipeline = Pipeline(name='Generic Pancreas Registration', description='Projecting the Generic Pancreas to the HRA Female Pancreas', params='../configs/params.yaml')

In [10]:
pipeline.params

{'nonrigid_registration': {'autotune': 'False',
  'beta': 0.5,
  'decimation': 'False',
  'distance_threshold': '1e-8',
  'lambda': 50,
  'max_iterations': 1000,
  'seed': 26},
 'rigid_registration': {'global_distance_threshold_factor': 0.23,
  'global_edge_length_threshold_factor': 0.95,
  'global_max_correspondence': 0.999,
  'global_max_iterations': 100000,
  'max_nn': 100,
  'refine_distance_threshold_factor': 0.1,
  'voxel_size': 0.025}}

In [11]:
# run registration
projections = pipeline.run(source=source, target=target)

In [12]:
 # display result
projections.registration.show()

#### Apply Projections to Tissue Blocks

In [ ]:
# load tissue blocks
atlas = trimesh.load('..data/Ovary/Source/generic_ovary_updated.glb')

# create dummy donor data (since this data is not available)
# ideally this information will be available in the RUI JSON being read from using from_sample method
donor = {'sex': 'Female', 
         'provider_name': 'TMC-Test', 
         'provider_uuid': 'Test-UUID', 
         'consortium_name': 'HuBMAP',
         'id': 'http://example.com/required/id', 
         'link': 'http://example.com/user/link'}

# convert to TissueBlock objects
blocks = [TissueBlock.from_geometry(block, donor=donor, target_name='VHFLeftOvary', label=label) for label, block in atlas.geometry.items()]

In [ ]:
# project
projected_blocks = [projections.project(block) for block in blocks]

#### Create Regsitration Object - Interfacing with the HRA Locations Processor Library

In [ ]:
from rui import RUIProcessor

In [ ]:
# initialize processor
processor = RUIProcessor(blocks=projected_blocks, registration_dir='../../VHFLeftKidney_to_VU500LeftKidney')

# initialize registration
processor.initialize_registration()

# generation RUIs
processor.generate_rui_locations()

#### Export Projections - Reproducibility and Future Use

In [ ]:
from dataclass import Projection

In [60]:
# export (remember to modify the save path)
projections.export(path='/Users/bhargavdesai/Desktop/IU/RA/CNS/Paper Results/Projections/Forward/VHMLeftKidney_to_VU500LeftKidney')

In [ ]:
# load (remember to modify the load path)
projections = Projection.load('/Users/bhargavdesai/Desktop/IU/RA/CNS/debug pipeline/GenericPancreasOBB_to_VHFPancreas')